In [41]:
import numpy as np
'''
Here using model free approach of using value to learn
this is called value learning(using value instead of q values in q learning)
'''

'\nHere using model free approach of using value to learn\nthis is called value learning(using value instead of q values in q learning)\n'

In [129]:
class Board:
  def __init__(self,player1,player2):
    self.board = np.zeros(9)
    self.player1 = player1
    self.player2 = player2
    self.end = False 
    self.player_to_play=1 

  def winner(self):
    if np.sum(self.board[0:3])==3 or np.sum(self.board[3:6])==3 or np.sum(self.board[6:9])==3:
      self.end=True
      return 1
    if np.sum(self.board[0:3])==-3 or np.sum(self.board[3:6])==-3 or np.sum(self.board[6:9])==-3: 
      self.end=True 
      return -1

    if (self.board[0]+self.board[3]+self.board[6]==3 or self.board[1]+self.board[4]+self.board[7]==3 or self.board[2]+self.board[5]+self.board[8]==3) :
      self.end=True 
      return 1
    
    if (self.board[0]+self.board[3]+self.board[6]==-3 or self.board[1]+self.board[4]+self.board[7]==-3 or self.board[2]+self.board[5]+self.board[8]==-3) :
      self.end=True 
      return -1

    if (self.board[0]+self.board[4]+self.board[8]==3 or self.board[6]+self.board[4]+self.board[2]==3) :
      self.end=True 
      return 1

    if (self.board[0]+self.board[4]+self.board[8]==-3 or self.board[6]+self.board[4]+self.board[2]==-3) :
      self.end=True 
      return -1

    if np.count_nonzero(self.board) == 9:
      self.end=True
      return 0

    self.end=False 
    return None
  
  def addposition(self,position):
    self.board[position]=self.player_to_play
    self.player_to_play*=-1

  def rewardfunc(self):
    result = self.winner()

    if result==1:
      self.player1.feedReward(1) 
      self.player2.feedReward(0) 
    
    elif result==-1:
      self.player1.feedReward(0)
      self.player2.feedReward(1) 
    
    else:
      self.player1.feedReward(0.2)
      self.player2.feedReward(0.5)

  def reset(self):
    self.board = np.zeros(9)
    self.end=False
    self.player_to_play=1 

  def train(self,round=10000):

    print("Training -> RL agent by making it play for ",round," games")

    for i in range(round):
      if i %1000 == 0 :
        print("currently playing round ",i)
      
      while not self.end:
        possible_moves=[]
        for j in range(9):
          if self.board[j]==0:
            possible_moves.append(j) 
        player1action = self.player1.pickAction(possible_moves,self.board,self.player_to_play)

        self.addposition(player1action)
        #self.printboard()
        self.player1.storestate(self.board)
        win = self.winner() 

        if win is not None:
          self.rewardfunc()
          self.player1.reset()
          self.player2.reset()
          self.reset()
          print(i," done")
          break
        else:
          possible_moves2=[]
          for j in range(9):
            if self.board[j]==0:
              possible_moves2.append(j) 
          player2action = self.player2.pickAction(possible_moves2,self.board,self.player_to_play)

          self.addposition(player2action)
          #self.printboard()
          self.player2.storestate(self.board)
          win = self.winner() 

          if win is not None:
            self.rewardfunc()
            self.player1.reset()
            self.player2.reset()
            self.reset()
            print(i," done" )
            break

  def playwithhuman(self):
    while not self.end:
      possible_moves=[]
      for j in range(9):
        if self.board[j]==0:
          possible_moves.append(j) 
      player1action = self.player1.pickAction(possible_moves,self.board,self.player_to_play)
      self.addposition(player1action)
      self.printboard()

      win=self.winner() 
      if win is not None:
     
        if win == 1:
          print(self.player2.name, "win!")
        else:
          print("tie")
        self.reset()
        break
      
    
      else:
        possible_moves2=[]
        for j in range(9):
          if self.board[j]==0:
            possible_moves2.append(j) 
        player2action = self.player2.pickAction(possible_moves2)

        self.addposition(player2action)
          #self.printboard()
        win = self.winner() 

        if win is not None:
          if win == -1:
            print(self.player2.name, "win!")
          else:
            print("tie")

          self.reset()

          break
   



  def printboard(self):
    for i in range(9):
      if i%3==0 :
        print()
      ch='_'
      if self.board[i]==1:
        ch='X'
      if self.board[i]==-1:
        ch='O'
      print(ch,end=' ')

In [118]:
class RLagent:
  def __init__(self,name,epsilon=0.35,alpha=0.2):
    self.states=[]
    self.name=name 
    self.alpha=alpha 
    self.epsilon = epsilon 
    self.gamma = 0.9 
    self.values = {}

  def pickAction(self,positions,current_board,symbol):
    #print(positions)
    #print(current_board)
    if np.random.uniform(0,1) <= self.epsilon:
      idx = np.random.choice(len(positions))
      action = positions[idx]
    else:
      value_max = -10000000
      for pos in positions:
        #print(current_board)
        boardcpy=np.empty_like(current_board)
        boardcpy[pos]=symbol 

        value = 0 if self.values.get(str(boardcpy)) is None else self.values.get(str(boardcpy))
        if value>=value_max :
          value_max= value
          action = pos 

    return action

  def feedReward(self,reward):
    for state in reversed(self.states):
      if self.values.get(str(state)) is None:
        self.values[str(state)]=0
      self.values[str(state)]+=self.alpha*(self.gamma*reward - self.values.get(str(state)))
      reward=self.values.get(str(state))

  def storestate(self,state):
    self.states.append(state)

  def reset(self):
    self.states=[]

  def policy(self):
    return self.values


In [119]:
class HumanPlayer:
  def __init__(self,name):
    self.name=name
  def pickAction(self,position):
    row, col = [int(x) for x in input("Enter position to place your coin - O (row, col) : ").split()] 
    action=3*row+col 
    return action

In [121]:
p1=RLagent('p1')
p2=RLagent('p2')
state=Board(p1,p2)
state.train()

Streaming output truncated to the last 5000 lines.
5004  done
5005  done
5006  done
5007  done
5008  done
5009  done
5010  done
5011  done
5012  done
5013  done
5014  done
5015  done
5016  done
5017  done
5018  done
5019  done
5020  done
5021  done
5022  done
5023  done
5024  done
5025  done
5026  done
5027  done
5028  done
5029  done
5030  done
5031  done
5032  done
5033  done
5034  done
5035  done
5036  done
5037  done
5038  done
5039  done
5040  done
5041  done
5042  done
5043  done
5044  done
5045  done
5046  done
5047  done
5048  done
5049  done
5050  done
5051  done
5052  done
5053  done
5054  done
5055  done
5056  done
5057  done
5058  done
5059  done
5060  done
5061  done
5062  done
5063  done
5064  done
5065  done
5066  done
5067  done
5068  done
5069  done
5070  done
5071  done
5072  done
5073  done
5074  done
5075  done
5076  done
5077  done
5078  done
5079  done
5080  done
5081  done
5082  done
5083  done
5084  done
5085  done
5086  done
5087  done
5088  done
5089  done
509

In [123]:
load=p1.policy()
print(load)

{'[ 1.  0.  1.  0.  1. -1. -1. -1.  1.]': 0.685517041450954, '[ 1.  0. -1.  1. -1.  1. -1.  1. -1.]': 0.0, '[ 1. -1.  1.  1. -1.  1. -1.  1. -1.]': 0.12668661019026717, '[-1.  1.  1. -1. -1.  1.  1. -1.  1.]': 0.6331156867300608, '[ 0.  1. -1. -1. -1.  1. -1.  1.  1.]': 0.0, '[ 0.  0.  1. -1. -1.  1.  1. -1.  1.]': 0.6857600839356478, '[ 0.  0. -1.  1.  1.  1. -1. -1.  1.]': 0.685656334238258, '[ 0.  0. -1.  0. -1.  1. -1.  1.  1.]': 0.0, '[ 0. -1. -1.  1. -1.  1.  1. -1.  1.]': 0.0, '[ 1.  0. -1.  1. -1.  0.  1.  0.  0.]': 0.17287200000000003, '[ 1. -1.  1.  1. -1. -1.  1.  1. -1.]': 0.6334330508716701, '[ 1.  0.  0.  1. -1.  1.  1. -1. -1.]': 0.6857600839051211, '[ 1.  1.  0. -1. -1. -1.  1. -1.  1.]': 0.0, '[ 0.  1. -1.  1. -1.  1. -1. -1.  1.]': 0.0, '[ 0.  0.  1. -1.  1. -1.  1. -1.  1.]': 0.6857600839356494, '[ 1.  1.  0.  0.  0.  1. -1. -1. -1.]': 0.0, '[ 1.  1.  1. -1.  1. -1.  1. -1. -1.]': 0.6334325146139773, '[ 1.  0.  0.  1. -1. -1.  1.  1. -1.]': 0.685760083935649, '[ 1.  

In [130]:
p2=HumanPlayer('me')
state=Board(p1,p2)
state.playwithhuman()


_ _ _ 
_ _ _ 
_ _ X Enter position to place your coin - O (row, col) : 1 0

_ _ _ 
O _ _ 
_ X X Enter position to place your coin - O (row, col) : 2 1

_ _ _ 
O _ _ 
X O X 

KeyboardInterrupt: ignored